In [1]:
%pwd

'e:\\Med GenAI\\MediAssist-AI-End-to-End-Medical-Chatbot\\research'

In [2]:
cd ..

e:\Med GenAI\MediAssist-AI-End-to-End-Medical-Chatbot


In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os
load_dotenv()

True

In [6]:
# from PyPDF2 import PdfReader
# def get_pdf_text(pdf_docs):
#     text=""
#     for pdf in pdf_docs:
#         pdf_reader= PdfReader(pdf)
#         for page in pdf_reader.pages:
#             text+= page.extract_text()
#     return  text



# def get_text_chunks(text):
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
#     chunks = text_splitter.split_text(text)
#     return chunks

In [4]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [5]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

In [7]:
# #Extract Data From the PDF File
# def load_pdf_file(data):
#     loader= DirectoryLoader(data,
#                             glob="*.pdf",
#                             loader_cls=PyPDFLoader)

#     documents=loader.load()

#     return documents

In [79]:
from pypdf import PdfReader
def extract_text_from_digitally_born_pdf_pypdf(pdf_path):
    text_content = ""
    try:
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            text_content += page.extract_text() + "\n" # Add a newline between pages
        return text_content
    except Exception as e:
        print(f"Error extracting text with pypdf: {e}")
        return None

In [81]:
book_text = extract_text_from_digitally_born_pdf_pypdf("Data/Medical-book.pdf")

In [7]:
extracted_data=load_pdf_file(data='Data/')
len(extracted_data)

637

In [8]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [ ]:
minimal_docs = filter_to_minimal_docs(extracted_data)
minimal_docs

AttributeError: 'str' object has no attribute 'metadata'

In [74]:
len(minimal_docs)

637

In [85]:
import google.generativeai as genai
model_for_token_counting = genai.GenerativeModel('gemini-2.0-flash') 

In [76]:
def count_gemini_tokens(text: str) -> int:
    """Counts tokens using the Gemini model's tokenizer."""
    return model_for_token_counting.count_tokens(text).total_tokens

In [86]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,  # Max tokens per chunk
    chunk_overlap=200, # Overlap between chunks
    length_function=count_gemini_tokens, # Use Gemini's tokenizer
    separators=["\n\n", "\n", " ", ""], # Default separators, good for general text
    # add_start_index=True # Optional: adds metadata for original position
)

In [87]:
chunks = text_splitter.split_text(book_text)

KeyboardInterrupt: 

In [88]:
print("Length of Text Chunks", len(chunks))

NameError: name 'chunks' is not defined

In [ ]:
#Split the Data into Text Chunks
def text_split(minimal_docs):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
    text_chunks=text_splitter.split_documents(minimal_docs)
    return text_chunks

In [90]:
text_chunks=text_split(book_text)
print("Length of Text Chunks", len(text_chunks))

AttributeError: 'str' object has no attribute 'page_content'

In [72]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    maxBatchSize= 80
)

In [64]:
result = embeddings.embed_query("Hello WOrld")
len(result), result

(768,
 [0.06716260313987732,
  -0.015745405107736588,
  -0.04832056164741516,
  -0.012914348393678665,
  0.022446652874350548,
  -0.008497313596308231,
  0.02000141143798828,
  0.0028713808860629797,
  0.007473421283066273,
  0.015388470143079758,
  0.07189052551984787,
  0.047062113881111145,
  0.005549346096813679,
  -0.06943696737289429,
  -0.013242038898169994,
  0.0014478419907391071,
  0.0065623377449810505,
  0.0061989654786884785,
  -0.0018574587302282453,
  -0.01572871021926403,
  0.002748878439888358,
  0.027116255834698677,
  -0.01792336255311966,
  -0.02341834455728531,
  0.026898030191659927,
  0.0010546112898737192,
  -0.00995844230055809,
  -0.07074843347072601,
  0.01392455492168665,
  0.06706225126981735,
  -0.03211153671145439,
  0.023601148277521133,
  -0.059650953859090805,
  0.0023785571102052927,
  0.03283407911658287,
  -0.06502316892147064,
  0.03287005051970482,
  0.02270207554101944,
  -0.005394322797656059,
  -0.008852369152009487,
  0.025906512513756752,
  -

In [65]:
from pinecone import ServerlessSpec

index_name = "medical-bot"  # change if desired

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

In [73]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Fri, 18 Jul 2025 12:13:05 GMT', 'Content-Type': 'application/json', 'Content-Length': '94', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '7108', 'x-pinecone-request-id': '6668664029263117632', 'x-envoy-upstream-service-time': '44', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Request size 3MB exceeds the maximum supported size of 2MB","details":[]}


New Way

In [8]:
from pypdf import PdfReader
def extract_text_from_digitally_born_pdf_pypdf(pdf_path):
    text_content = ""
    try:
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            text_content += page.extract_text()
        return text_content
    except Exception as e:
        print(f"Error extracting text with pypdf: {e}")
        return None

In [9]:
book_text = extract_text_from_digitally_born_pdf_pypdf("Data/Medical-book.pdf")

In [24]:
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
    chunks = text_splitter.split_text(text)
    return chunks

In [25]:
text_chunks = get_text_chunks(book_text)

In [26]:
len(text_chunks)

9585

In [27]:
text_chunks[0]

'The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITIONThe GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B\n1STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor\nChristine B. Jeryan, Managing Editor'

In [33]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)

In [15]:
result = embeddings.embed_query("Hello WOrld")
len(result)

768

In [32]:
from pinecone import ServerlessSpec

index_name = "medical-bot"  # change if desired

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

In [39]:
index = pc.Index(index_name)  
index.describe_index_stats()  

{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [41]:
len(book_text)

2630005

In [42]:
for batch in book_text.iter_documents(batch_size=100):  
    index.upsert(batch) 

AttributeError: 'str' object has no attribute 'iter_documents'

In [114]:
text_chunks[15:500]

['New York, NY\nLinda Chrisman\nMassage Therapist and Educator\nOakland, CA\nLisa Christenson, Ph.D.\nScience Writer\nHamden, CT\nGeoffrey N. Clark, D.V .M.\nEditor\nCanine Sports Medicine Update\nNewmarket, NH\nGALE ENCYCLOPEDIA OF MEDICINE 2 XIII\nCONTRIBUTORSRhonda Cloos, R.N.\nMedical Writer\nAustin, TX\nGloria Cooksey, C.N.E\nMedical Writer\nSacramento, CA\nAmy Cooper, M.A., M.S.I.\nMedical Writer\nVermillion, SD\nDavid A. Cramer, M.D.\nMedical Writer\nChicago, IL\nEsther Csapo Rastega, R.N., B.S.N.\nMedical Writer\nHolbrook, MA\nArnold Cua, M.D.\nPhysician\nBrooklyn, NY\nTish Davidson, A.M.\nMedical Writer\nFremont, California\nDominic De Bellis, Ph.D.\nMedical Writer/Editor\nMahopac, NY\nLori De Milto\nMedical Writer\nSicklerville, NJ\nRobert S. Dinsmoor\nMedical Writer\nSouth Hamilton, MA\nStephanie Dionne, B.S.\nMedical Writer\nAnn Arbor, MI\nMartin W. Dodge, Ph.D.\nTechnical Writer/Editor\nCentinela Hospital and Medical\nCenter\nInglewood, CA\nDavid Doermann\nMedical Writer\n

In [34]:
# Embed each chunk and upsert the embeddings into your Pinecone index.

from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [19]:
print(f"Adding {len(text_chunks)} chunks to Pinecone in batches...")

Adding 5693 chunks to Pinecone in batches...


In [20]:
len(text_chunks[0])

468

In [21]:
# Set a reasonable character limit to check against.
# A very large chunk might be over 50,000 characters.
max_char_length = 50000 
problem_found = False

print("Checking for oversized text chunks...")
for i, chunk in enumerate(text_chunks):
    if len(chunk) > max_char_length:
        print(f"⚠️  WARNING: Chunk at index {i} is extremely large: {len(chunk)} characters.")
        problem_found = True

if not problem_found:
    print("✅ No chunks exceeded the test character length.")
    print("If the error persists, try lowering max_char_length and run this check again.")


Checking for oversized text chunks...
✅ No chunks exceeded the test character length.
If the error persists, try lowering max_char_length and run this check again.


In [38]:
docsearch.add_texts(
    texts=text_chunks
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Fri, 18 Jul 2025 18:26:17 GMT', 'Content-Type': 'application/json', 'Content-Length': '94', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '15477', 'x-pinecone-request-id': '7126104039989177826', 'x-envoy-upstream-service-time': '48', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Request size 3MB exceeds the maximum supported size of 2MB","details":[]}


In [36]:
# Replace "your-index-name" with the actual name of your index
index_details = pc.describe_index(index_name)

# Print the pod type (e.g., 's1.x1', 'p1.x1', etc.)
print(f"Index Pod Type: {index_details.pod_type}")

Index Pod Type: None


In [ ]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_texts(
    texts=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Fri, 18 Jul 2025 17:28:12 GMT', 'Content-Type': 'application/json', 'Content-Length': '118', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '33849', 'x-pinecone-request-id': '4917824011130072069', 'x-envoy-upstream-service-time': '2', 'server': 'envoy'})
HTTP response body: {"code":11,"message":"Error, message length too large: found 4878571 bytes, the limit is: 4194304 bytes","details":[]}
